# Assignment 1-1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow the python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [http://www.sfu.ca/computing/people/faculty.html](http://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [2]:
# Import Statements
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

#  Downloading Web page
web_url = "http://www.sfu.ca/computing/people/faculty.html"
downloaded_page = requests.get(web_url) 

# Saving the webpage as .html file
file_obj = open('csfaculty.html', 'wb')
file_obj.write(downloaded_page.content)
file_obj.close()

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [4]:
# Creating data frame to store parsed values
faculty_dataframe = pd.DataFrame(columns=['name', 'rank', 'area', 'profile', 'homepage'])

# Additional Variables
faculty_row_value = {}
profile_url_prefix = "http://www.sfu.ca"

# Extracting Relevant Content
with open('csfaculty.html', 'r') as page:
    content = page.read()
    soup = BeautifulSoup(content, 'html.parser')
    
    faculty_list = soup.find_all('div', attrs = {'class':'textimage section'})
    
    # Looping through each elements in the faculty list 
    for value in faculty_list:
        
        faculty_obj = value.find('div',  attrs = {'class':'text'})
        faculty_header = faculty_obj.h4.text.split(',')
        
        # Extracting Name and Rank of the Professor
        faculty_row_value['name'] = faculty_header[0].strip().title()
        faculty_row_value['rank'] = faculty_header[1].split('\n')[0].strip().title()

        faculty_detail = faculty_obj.find_all('p')
        record_length = len(faculty_detail)
        
        # Extracting Area of the Professor
        if record_length > 0 and faculty_detail[0] is not None:
            faculty_row_value['area'] = faculty_detail[0].text.split(':')[1].strip()
        else:
            faculty_row_value['area'] = None
        
        # Extracting Profile Link of the Professor
        if record_length > 0 and faculty_detail[1] is not None:
            faculty_row_value['profile'] = profile_url_prefix + faculty_detail[1].a['href'].replace(profile_url_prefix, "")
        else:
            faculty_row_value['profile'] = None
            
        # Extracting Home Page of the Professor
        faculty_homepage = faculty_detail[1].find_all('a') if record_length > 0 else []

        if len(faculty_homepage) >= 2:
                
            homepage_value = faculty_homepage[1].get('href')
                
            if homepage_value is not None and not homepage_value.startswith("http"):
                faculty_row_value['homepage'] = profile_url_prefix + homepage_value
            else:
                faculty_row_value['homepage'] = homepage_value
                    
        else:
            faculty_row_value['homepage'] = None
        
        # Appending the record as a row to the data frame     
        faculty_dataframe = faculty_dataframe.append(faculty_row_value, ignore_index=True) 

# Writing the Structured data to csv format
faculty_dataframe.to_csv('faculty_table.csv', index=False)

### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What interesting findings can you get from these visualizations? 

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

Below are some examples:

**Finding 1:** Professor# (26) is more than 2x larger than Associate Professor# (10). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** The Homepage has 22% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are interested in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2021+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2021+23-1990 = 54. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 65 profile pages and save each page as a text file. 

In [5]:
file_obj = None
faculty_df = pd.read_csv('faculty_table.csv')
profile_df = faculty_df[['name', 'profile']]
page_list = []

# Downloading and Saving each pages as text file
for row in profile_df.itertuples(index = False):
    file_name = row[0].replace(" ", "-") + ".txt"
    if not pd.isnull(row[1]):
        profile_page = requests.get(row[1]) 
        file_obj = open(file_name, 'wb')
        file_obj.write(profile_page.content)
        page_list.append(file_name)
        
file_obj.close()

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [12]:
# Additional Variables
faculty_grad_df = pd.DataFrame(columns=['name', 'gradyear'])
grad_info = {'name': None, 'gradyear': None}

education_section = None
prof_details= None

# Additional Methods
def extract_year(list_tag):
    edu_list = list_tag.find_all('li')
    year_value = []
    
    for education in edu_list:
        year = re.findall(r'\d+', education.text)
        for i in year:
            year_value.append(i)
            
    if len(year_value) > 0:
        grad_info['gradyear'] = min(map(int, year_value))
        
def get_text_section(prof_section):
    return prof_section.find_all('div', class_ = "text parbase section")

# Extracting the Earliest graduation year
for pro_page in page_list:
    content = open(pro_page,'r',encoding="utf-8").read()
    soup = BeautifulSoup(content, 'html.parser')
    
    # Extracting the Professor Name
    name_tag = soup.find('div',  attrs = {'class':'title section'})
    grad_info['name'] = name_tag.div.h1.text.strip().title()
    
    # Extracting the Graduation year
    prof_details = soup.find('div',  attrs = {'class':'parsys_column cq-colctrl-lt0-c0'})
    if prof_details is not None: 
        education_section = get_text_section(prof_details)
        list_tag = None
        
        if len(education_section) > 1:
            list_tag = education_section[1].div.ul
        
        if list_tag is not None:
            extract_year(list_tag)
          
    # Case 2
    prof_details = soup.find('div',  attrs = {'class':'parsys_column cq-colctrl-lt0-c1'})
    if prof_details is not None:
        education_section = get_text_section(prof_details)
        list_tag = education_section[0].div.ul
        
        if list_tag is not None:
            extract_year(list_tag)
            
        else:
            list_tag = education_section[0].div.p
            if list_tag is not None and grad_info['gradyear'] is None:
                year_list = re.findall(r'\d+', list_tag.text)
                
                if len(year_list) > 0:
                    grad_info['gradyear'] = min(map(int, year_list))

    # Case 3
    prof_details = soup.find('div',  attrs = {'class':'parsys_column cq-colctrl-lt4-c1'})
    if prof_details is not None: 
        education_section = get_text_section(prof_details)
        list_tag = education_section[0].div.ul
        extract_year(list_tag)
      
    # Appending the extracted values to data frame
    faculty_grad_df = faculty_grad_df.append(grad_info, ignore_index=True)
    grad_info['name'], grad_info['gradyear'] = None, None

# Saving the data frame as csv file
faculty_grad_df.to_csv('faculty_grad_year.csv', index = True)

### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2021+23-df["gradyear"]

plot(df, "age")

## Submission

Complete the code in this notebook, and submit it to the CourSys activity `Assignment 1`.

## Bonus: Contribute to DataPrep

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature) at [link1](https://github.com/sfu-db/dataprep/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 13, 2022`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star-2022.png" width="1000">